# Run 1.1p Object Tables with Dask+Holoviews
<br>Owner: **Michael Wood-Vasey** ([@wmwv](https://github.com/LSSTDESC/DC2-analysis/issues/new?body=@wmwv))
<br>Last Verified to Run: **2018-10-11**
    
* Demonstrate the use of the DPDD-style Object table in Parquet format. 
* Uses Dask+Holoviews+Datashader to demonstrate visualizing the entire dataset.

Learning Objectives:
After completing and studying this Notebook, a person should be able to
1. Load a Parquet file with Dask.
    - Understand that specifying the columns to select increasing performance.
2. Make a plot using Holoviews
3. Use Datashader to interactively rasterize a large dataset for display.

Logistics:

1. These tests were conducted on NERSC through the https://jupyter-dev.nersc.gov interface.

2. Requires:
```
holoviews
datashader
bokeh
```

These were pip-installed locally with `--user`.

3. This was run using a custom kernel specification
`/global/homes/w/wmwv/.local/share/jupyter/kernels/desc-python-dask`
  - This custom kernel specification was used to control the OMP_NUM_THREADS setting.
  - Tests were run with `OMP_NUM_THREADS=4`, but I don't believe there's any real multi-threading.

Current Status:
* Quick demonstration for people who want to check out the Parquet file and learn.
* Future work planned to demonstrate selection, brushing, and simple outlier identification.

References:  
    https://dask.org  
    https://datashader.org  
    https://parquet.apache.org  
    https://holoviews.org  

In [ ]:
import os

import dask.dataframe as dd
import datashader as ds
import holoviews as hv
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize

In [ ]:
hv.extension('bokeh')

## Define and (lazy-)load our data

In [ ]:
tract = 4850

data_dir = '/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/summary'

datafile_tract = os.path.join(data_dir, 'dpdd_object_tract_%d.parquet' % tract)
datafile_all = os.path.join(data_dir, 'dpdd_object.parquet')

In [ ]:
# Switch between these to go from analyzing one tract ('datafile_tract'), which renders in seconds
# to the entire set of 20 tracts ('datafile_all'), which renders in minutes.
datafile = datafile_tract
# datafile = datafile_all

# Specify the columns we need.  This allows for significant performance advantages when reading a column-based storage format such as Parquet.
df = dd.read_parquet(datafile, columns=['ra', 'dec', 'mag_g', 'mag_r', 'mag_i',
                                        'magerr_g', 'magerr_r', 'magerr_i', 'extendedness'])

In [ ]:
print(df.columns)

## Clean and Create Color Columns

In [ ]:
# Clean
snr_magerr_threshold = 0.3  # mag
df = df[(df.magerr_g < snr_magerr_threshold) & (df.magerr_r < snr_magerr_threshold) & (df.magerr_i < snr_magerr_threshold)]

In [ ]:
df['g-r'] = df['mag_g'] - df['mag_r']
df['r-i'] = df['mag_r'] - df['mag_i']

In [ ]:
gal = df[(df.extendedness > 0.95)]
star = df[(df.extendedness < 0.95)]

## Create HoloViews `Points` objects and Wrap with Datashader

Create for position, magnitude and color.  Use datashader to provide rasterized images that display in finite time but are still zoomable and will re-raster at different sizes.

In [ ]:
points_ra_dec = hv.Points(df, kdims=['ra', 'dec'], vdims=['g-r'])
ra_dec = datashade(points_ra_dec, aggregator=ds.mean('g-r'))

In [ ]:
points_mag_magerr = hv.Points(df, kdims=[hv.Dimension('mag_g', soft_range=(14, 28)),
                                         'magerr_g'])
mag_magerr = datashade(points_mag_magerr)

In [ ]:
points_color_mag = hv.Points(df, kdims=[hv.Dimension('g-r', soft_range=(-2, 3)),
                                        hv.Dimension('mag_g', soft_range=(28, 14))])
color_mag = datashade(points_color_mag)

In [ ]:
points_color_color = hv.Points(df, kdims=[hv.Dimension('g-r', soft_range=(-2, 3)),
                                          hv.Dimension('r-i', soft_range=(-2, 3))])
# color_color = datashade(points_color_color.hist(dimension=['g-r', 'r-i']))
color_color = datashade(points_color_color)

In Holoviews one uses the `+` operator to put these three different visualizations next to each other.

In [ ]:
ra_dec + color_mag + color_color

The color-magnitude and color-color plots will zoom together.  This based on the ranges of the dimensions; it's not subsetting the points in the view.

No I don't know how to reverse the magnitude axis so that brighter is up.  I spent an hour trying to figure that out.

## Datashade Multiple Sets

Show separate by `extendedness` parameter.  Note that `extendedness` is pretty conservative with a high false negative at faint magnitudes.

In [ ]:
gal_mag_magerr = hv.Points(gal, kdims=[hv.Dimension('mag_g', soft_range=(14, 28)),
                                       'magerr_g'])
gal_color_mag = hv.Points(gal, kdims=[hv.Dimension('g-r', soft_range=(-2, 3)),
                                      hv.Dimension('mag_g', soft_range=(28, 14))])
gal_color_color = hv.Points(gal, kdims=[hv.Dimension('g-r', soft_range=(-2, 3)),
                                        hv.Dimension('r-i', soft_range=(-2, 3))])

star_mag_magerr = hv.Points(star, kdims=[hv.Dimension('mag_g', soft_range=(14, 28)),
                                         'magerr_g'])
star_color_mag = hv.Points(star, kdims=[hv.Dimension('g-r', soft_range=(-2, 3)),
                                        hv.Dimension('mag_g', soft_range=(28, 14))])
star_color_color = hv.Points(star, kdims=[hv.Dimension('g-r', soft_range=(-2, 3)),
                                          hv.Dimension('r-i', soft_range=(-2, 3))])

typed_mag_magerr = {'gal': gal_mag_magerr, 'star': star_mag_magerr}
typed_color_mag = {'gal': gal_color_mag, 'star': star_color_mag}
typed_color_color = {'gal': gal_color_color, 'star': star_color_color}

shaded_mag_magerr = datashade(hv.NdOverlay(typed_mag_magerr, kdims='type'),
                              aggregator=ds.count_cat('type'))
shaded_color_mag = datashade(hv.NdOverlay(typed_color_mag, kdims='type'),
                             aggregator=ds.count_cat('type'))
shaded_color_color = datashade(hv.NdOverlay(typed_color_color, kdims='type'),
                               aggregator=ds.count_cat('type'))

In [ ]:
shaded_mag_magerr + shaded_color_mag + shaded_color_color

The galaxies are red, while the "stars" (== not obviously extended) are blue.

Zoom in to mag - magerr plot to see the outlying cluster of higher uncertainties as a function of magnitude are galaxies.

Note:  
Constructing a legend for the above is unfortunately a little unobvious and awkward.  We lost the information when we datashaded the NdOverlay.  You could do it by creating a new set of empty NdOverlay object to get the colors. 

This is relying on the fact that the above commands used the default color map.  
Explicitly specifying the color above would have been better.
```
from datashader.colors import Sets1to3 # default datashade() and shade() color cycle
color_key = {k: Sets1to3[i] for i, k in enumerate(typed_color_mag)}
color_points = hv.NdOverlay({k: hv.Points([gal['g-r'][0], gal['mag_g'][0]],
                                          label=str(k)).options(color=v) for k, v in color_key.items()})
                                          
shaded_color_mag * color_points + shaded_color_color * color_points
```

Above code adapted from http://holoviews.org/user_guide/Large_Data.html

## Use hover-over aggregation

We can set up a dynamic hover-over that gives information about the local area.  In this case we're just doing a count of the number of points in a given rectangular region.

Note the use of the `*` to compose the results of `datashade` and `hv.util.Dynamic`.  This is the idiom in Holoviews to combine several different visualizations/tools.

In [ ]:
from holoviews.streams import RangeXY

# A funnily-named wrapper function to generate hover-overs by count.
def dynamate(points, width=400, height=400, nx=10, ny=10):
    """Datashades points at width, height.  Hover-over in dynamic boxes of nx x ny at given display size"""
    datashaded_points = datashade(points, width=width, height=height)
    hover_over_count = \
        hv.util.Dynamic(rasterize(points_mag_magerr, width=nx, height=ny, streams=[RangeXY]),
                        operation=hv.QuadMesh)
    return datashaded_points * hover_over_count

In [ ]:
%%opts QuadMesh [tools=['hover']] (alpha=0 hover_alpha=0.2)
dynamic_mag_magerr = dynamate(points_mag_magerr)
dynamic_color_mag = dynamate(points_color_mag)
dynamic_color_color = dynamate(points_color_color)

In [ ]:
dynamic_mag_magerr + dynamic_color_mag + dynamic_color_color

In [ ]:
len(df)

In [ ]:
hv.help(color_mag)